#### 1. **Introduction to Association Rules**

Association rules are widely used in market basket analysis to discover relationships between items in transactional data. The goal is to find interesting relationships (or "rules") that imply the co-occurrence of items. 

- **Antecedent (LHS)**: The item(s) on the left-hand side of the rule.
- **Consequent (RHS)**: The item(s) on the right-hand side of the rule.

An example rule:  
**{Milk, Bread} → {Butter}**

This means that if a customer buys *Milk* and *Bread*, they are likely to buy *Butter* as well.

---

#### 2. **Metrics for Association Rules**

To evaluate the strength of association rules, we use three key metrics:

1. **Support**:  
   The proportion of transactions in which the item(s) appear.  
  $$
   Support(A → B) = \frac{\text{Number of transactions containing both A and B}}{\text{Total number of transactions}}
  $$

2. **Confidence**:  
   The probability that B is purchased given A is already purchased.  
  $$
   Confidence(A → B) = \frac{\text{Support(A and B)}}{\text{Support(A)}}
  $$

3. **Lift**:  
   The ratio of the observed support to the expected support if A and B were independent. Lift shows the strength of the rule beyond random chance.  
  $$
   Lift(A → B) = \frac{\text{Confidence(A → B)}}{\text{Support(B)}}
  $$

---

#### 3. **Step-by-Step Example with Numbers**

Let’s assume a small dataset with 5 transactions:

| Transaction ID | Items Bought               |
|----------------|----------------------------|
| 1              | Milk, Bread, Butter         |
| 2              | Milk, Bread                 |
| 3              | Bread, Butter               |
| 4              | Milk                        |
| 5              | Milk, Bread, Butter, Cheese |

We are interested in the association rule:  
**{Milk, Bread} → {Butter}**

- **Step 1**: Calculate **Support** of the rule:  
$$
  Support(Milk, Bread → Butter) = \frac{\text{Transactions with Milk, Bread, and Butter}}{\text{Total Transactions}} = \frac{2}{5} = 0.4
$$

- **Step 2**: Calculate **Confidence** of the rule:  
$$
  Confidence(Milk, Bread → Butter) = \frac{\text{Transactions with Milk, Bread, and Butter}}{\text{Transactions with Milk and Bread}} = \frac{2}{3} ≈ 0.67
$$

- **Step 3**: Calculate **Lift** of the rule:  
  First, we need the support of Butter:
$$
  Support(Butter) = \frac{\text{Transactions with Butter}}{\text{Total Transactions}} = \frac{3}{5} = 0.6
$$
  Then calculate Lift:
$$
  Lift(Milk, Bread → Butter) = \frac{\text{Confidence(Milk, Bread → Butter)}}{\text{Support(Butter)}} = \frac{0.67}{0.6} ≈ 1.11
$$

---

#### 4. **Python Code Example**

Here’s a Python implementation using the `mlxtend` library, which is great for generating frequent itemsets and association rules.

**Explanation**:
- **Step 1**: We create a binary matrix of transactions where each item is represented by 1 (purchased) or 0 (not purchased).
- **Step 2**: The Apriori algorithm identifies frequent itemsets (those that meet a minimum support threshold).
- **Step 3**: Association rules are generated based on the frequent itemsets, with a minimum confidence threshold.


In [6]:
# Install the necessary package
# pip install mlxtend

import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Step 1: Create the transaction dataset
data = {'Milk': [1, 1, 0, 1, 1],
        'Bread': [1, 1, 1, 0, 1],
        'Butter': [1, 0, 1, 0, 1],
        'Cheese': [0, 0, 0, 0, 1]}

# Convert the integer values to boolean (True/False) to eleminate deprecation warning message
df = pd.DataFrame(data).astype(bool)
df

,Milk,Bread,Butter,Cheese
0,True,True,True,False
1,True,True,False,False
2,False,True,True,False
3,True,False,False,False
4,True,True,True,True


In [7]:
# Step 2: Apply Apriori algorithm to find frequent itemsets
# min_support=0.4 means we only consider itemsets with support of at least 40%
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)

# Step 3: Generate association rules with a minimum confidence of 0.6
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

# Display the association rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

       antecedents    consequents  support  confidence      lift
0           (Milk)        (Bread)      0.6    0.750000  0.937500
1          (Bread)         (Milk)      0.6    0.750000  0.937500
2         (Butter)         (Milk)      0.4    0.666667  0.833333
3          (Bread)       (Butter)      0.6    0.750000  1.250000
4         (Butter)        (Bread)      0.6    1.000000  1.250000
5    (Milk, Bread)       (Butter)      0.4    0.666667  1.111111
6   (Milk, Butter)        (Bread)      0.4    1.000000  1.250000
7  (Bread, Butter)         (Milk)      0.4    0.666667  0.833333
8         (Butter)  (Milk, Bread)      0.4    0.666667  1.111111


#### Interpreting the Results
##### Columns in the Results

1. **Antecedents**: The item(s) on the left-hand side (LHS) of the rule. These are the "if" conditions.
2. **Consequents**: The item(s) on the right-hand side (RHS) of the rule. These are the "then" conditions.
3. **Support**: The proportion of transactions that contain both the antecedents and the consequents.
4. **Confidence**: The probability that the consequent(s) will occur given that the antecedent(s) have occurred.
5. **Lift**: The strength of the rule relative to random chance. If the lift is greater than 1, it indicates a positive correlation between the antecedents and consequents.

#### Interpreting Each Rule

Let's break down some of the rows to see how they can be interpreted:

##### Rule 0: **{Milk} → {Bread}**

- **Support**: 0.6  
  This means that 60% of all transactions include both *Milk* and *Bread*.
  
- **Confidence**: 0.75  
  This means that when *Milk* is purchased, there is a 75% chance that *Bread* will also be purchased.
  
- **Lift**: 0.9375  
  Since the lift is less than 1, it indicates that buying *Milk* slightly *decreases* the likelihood of buying *Bread* compared to chance. The rule might not be particularly strong or valuable.

##### Rule 3: **{Bread} → {Butter}**

- **Support**: 0.6  
  This means that 60% of transactions contain both *Bread* and *Butter*.
  
- **Confidence**: 0.75  
  If a customer buys *Bread*, there is a 75% chance that they will also buy *Butter*.
  
- **Lift**: 1.25  
  A lift greater than 1 suggests a positive correlation between *Bread* and *Butter*, meaning customers are more likely to buy *Butter* when they buy *Bread* than by random chance.

##### Rule 5: **{Milk, Bread} → {Butter}**

- **Support**: 0.4  
  This means that 40% of all transactions include *Milk*, *Bread*, and *Butter* together.
  
- **Confidence**: 0.67  
  When a customer buys both *Milk* and *Bread*, there is a 67% chance they will also buy *Butter*.
  
- **Lift**: 1.11  
  Since the lift is greater than 1, it indicates a positive correlation between buying *Milk* and *Bread* together and also buying *Butter*. The rule is somewhat useful but not extremely strong.

#### General Interpretation Guidelines

- **High Support**: A rule with high support occurs frequently in the dataset, making it potentially useful for common patterns (but may be too general if support is too high).
  
- **High Confidence**: Rules with high confidence are good predictors—when the antecedent occurs, the consequent is likely to occur as well.

- **Lift Greater Than 1**: A lift value greater than 1 indicates a strong association between the antecedent and consequent. The higher the lift, the stronger the association.

By combining these metrics, you can determine which rules are the most meaningful. In practice, you want rules with relatively high support, confidence, and lift to make effective business decisions.

#### Example Application:

- **{Bread} → {Butter}** could be used for targeted promotions. Since customers who buy *Bread* are more likely to buy *Butter* (with a lift of 1.25), you might offer a discount on *Butter* to customers purchasing *Bread*.

#### 5. **Conclusion**

Association rules help to find meaningful relationships between items in large datasets, making them extremely valuable in areas like market basket analysis. The key metrics (support, confidence, and lift) offer a clear way to evaluate the strength of any given rule. Using tools like Python and the `mlxtend` library, you can quickly generate and analyze association rules from transaction data.

**Homework**:  
Analyze a given dataset and generate at least 3 strong association rules using a minimum support of 0.5 and a confidence of 0.7. Interpret the results and explain how they could be applied in a real-world business scenario.